# Example 3 - Low CMC Surfactant Design

Surfactant molecules play important roles in a wide variety of disciplines of study, from lubricants
and coating to pharmaceuticals and drug delivery systems. This wide applicability of study is
due to the role of surfactant molecules act as compatibilizers between dissimilar materials phases.
While there are many metrics that are used to characterize surfactant molecules, the most common
is the critical micelle concentration (CMC). CMC is traditionally the experimentally determined
concentration at which individual surfactant molecules will self-assemble into larger aggregates
(micelles). This value is critically important as the desirable properties of surfactants (solubilizing
differing phases, enabling biocompatibility etc.) are typically only enabled when the solution
concentration of the surfactant is above the CMC. To design surfactant molecules with a desired
CMC, the task is often challenging and relies heavily on domain-knowledge based expertise. Hence,
the design task of minimizing CMC is both well-suited for an LLM agent, and a desirable objective
to reduce the reliance on domain expertise for chemical synthesis.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
import dziner
import os
from langchain._api import LangChainDeprecationWarning
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
os.environ["ANTHROPIC_API_KEY"] = 'YOUR_ANTHROPIC_API_KEY'

## Defining Customized Agent Tools

### Setting up the CMC predictor

This model is based on the work of [Qin et al.](https://pubs.acs.org/doi/10.1021/acs.jpcb.1c05264) without any adaptations. The authors used a
Graph Convolutional Neural Network (GCN) to predict the critical micelle concentration (CMC)
of surfactants based on their molecular structure as SMILES input. The CMC training data were
experimentally measured at room temperature (between 20 and 25 °C) in water for 202 surfactants
coming from various classes, including nonionic, cationic, anionic, and zwitterionic. The model
architecture leverages graph convolutional layers to process molecular graphs, where atoms are
represented as nodes and bonds as edges, effectively capturing both topological and constitutional
information. The GCN includes average pooling to aggregate atom-level features into a fixed-size
graph-level vector, followed by fully connected layers with ReLU activations for the final regression
of the log CMC value. In terms of performance, the GCN has a root-mean-squared-error (RMSE)
of 0.23 and an R2 of 0.96 on test data for nonionic surfactants, outperforming previous quantitative
structure-property relationship (QSPR) models. The ensemble of models used in our study are based
on an 11-fold cross-validation with mean RMSE of 0.32. For more details on the model, refer to
reference
See [paper](https://pubs.acs.org/doi/10.1021/acs.jpcb.1c05264) for more details on the surrogate model.

In [2]:
## And is coming from this URL 
## https://github.com/zavalab/ML/blob/master/CMC_GCN/saved_models/gnn_logs_save_202_hu256_lr0.005_best_trainalles_seed4592/ep200bs5lr0.005hu256es.pth.tar

from langchain.agents import tool
import torch
from dziner.surrogates.CMC.model import predict_single_point
import numpy as np

@tool
def predict_cmc(smiles):
    '''
    This tool predicts the mean log Critical Micelle Concentration (LogCMC) and it standard deviation for a SMILES.
    The model is based on a GCN from this paper:
    https://pubs.acs.org/doi/abs/10.1021/acs.jpcb.1c05264
    '''
    base_model_path = "../dziner/surrogates/CMC/training/11-folds/ep200bs5lr0.005kf11hu256cvid{}.pth.tar"
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    predictions = []

    # Looping through the ensemble of models with 11-fold cross validation.
    for cvid in range(11):  
        model_path = base_model_path.format(cvid)
        try:
            prediction = predict_single_point(smiles, model_path, device)
            predictions.append(prediction)
        except:
            predictions.append(None)
    
    valid_predictions = [p for p in predictions if p is not None]
    
    if not valid_predictions:
        return 'Invalid molecule', None
    
    mean_prediction = np.mean(valid_predictions)
    std_prediction = np.std(valid_predictions)
    
    return np.round(mean_prediction, 3), np.round(std_prediction, 3)


### Retrieving domain-knowledge from scientific literature

We use obtain some domain-knowledge on how the presence of different amino acids can affect the hemolytic activity within the peptide sequence.

In [3]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
Embedding_model = 'text-embedding-3-large' 

RetrievalQA_prompt = """What are the design guidelines for making the surfactant with a Critical Micelle Concentration? 
    This can be based on making changes to the functional groups or other changes to the molecule. Summarize your answer and cite paper
    by the title, DOI and the journal and year the document was published on."""

@tool
def domain_knowledge(prompt):
    '''Useful for getting chemical intuition for the problem.
     This tool looks up design guidelines for molecules with lower Critical Micelle Concentration by looking through research papers.
    It also includes information on the paper citation or DOI.
    '''
    guide_lines = []
    for m in range(6):
        text_splitter = CharacterTextSplitter(
            chunk_size=500, chunk_overlap=20)
        paper_file =f'../data/papers/CMC/{m}.pdf'
        pages = PyPDFLoader(paper_file).load_and_split()
        sliced_pages = text_splitter.split_documents(pages)
        faiss_vectorstore = FAISS.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))    
        llm=ChatOpenAI(
                        model_name='gpt-4o',
                        temperature=0.1,
                        )
        g = dziner.RetrievalQABypassTokenLimit(faiss_vectorstore, RetrievalQA_prompt, llm)
        guide_lines.append(g)
    return " ".join(guide_lines)

## Defining dZiner agent

We use Claude 3.5 Sonnet to drive our agent.

In [4]:
from dziner.tools import mol_chemical_feasibility

tools = [domain_knowledge, mol_chemical_feasibility, predict_cmc]

tool_names = [tool.name for tool in tools]  
tool_desc = [tool.description for tool in tools]


initial_surfactant = "CCCCCCCCCC(=O)CC(=O)NC1CCOC1=O" 
Human_prompt = f"Make changes to {initial_surfactant} so it will have a lower Critical Micelle Concentration (CMC). Suggest 5 new candidates."

input_data = {
            "input": Human_prompt,
            "prompt": RetrievalQA_prompt,
            "tools": tools,
            "tool_names": tool_names,
            "tool_desc": tool_desc
        }

### Defining custom system messages for the agent

### Adding Design Constraints

You can add design design contraints to the model in natural language text. Here, we add one to keep the molecular weight of suggested new candidates lower than 600 g/mol, to ensure more realistic molecules

In [5]:
FORMAT_INSTRUCTIONS = """
Use the following format:

    Thought: you should always think about what to do and never leave this empty
    Action: the action to take, should be one of the tools [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I know the final answer (This is after you have completed generating 10 SMILES in your final answer)
    Final Answer: the final answer to the input question. 
    The Final Answer MUST come in JSON format with NO extra text.
    
    Example final answer format:

     "Iteration": 0,
      "SMILES": "CC(=O)OC1=CC=C(C=C1)C(=O)N2CCN(CC2)CC3=CC=CC=C3I",
      "Modification": The change made to the SMILES in the previous iteration and the detailed reasoning behind it.
      "logCMC": -9.8,
      "uncertainty": 0.1,
      "SA Score": 2.01,
      "Molecular Weight": 150,
      "Validity": "Valid"

    Report all iterations, even if the SMILES is invalid. An iteration includes, suggesting a new SMILES, determining its validity, computing its logCMC.
    
    Use the exact SMILES without any newlines or extra parentheses. Do not repeat SMILES that already exist in the iteration history.
    
    You have a final answer once 10 new SMILES candidates are generated and evaluated. 
       

"""

SUFFIX = """Start by:

1. Researching Design Guidelines: Summarize the guidelines in a few bullet points, with citations (e.g., paper DOI) included for any design guidelines used.

2. Evaluating Initial SMILES:

logCMC: Assess the logCMC of the initial SMILES.
Validity: Check the validity of the initial SMILES structure.
3. Modifying SMILES:

Manual Adjustments: Based on the guidelines from Step 1, make manual changes to the SMILES. These changes may involve adding, removing, or replacing elements, functional groups, or rings in the molecule’s core.
Apply different modifications that could enhance logCMC without significantly increasing molecular size/weight.
Alignment with Guidelines: Justify in great details how each new SMILES aligns with the design guidelines, such as adding aromatic rings if suggested.
4. Evaluating Modified SMILES:

logCMC: Assess the logCMC of each new SMILES.
Validity: Ensure each new SMILES is valid.

Choose the SMILES with the lowest logCMC.
If no improvement is seen, revert to the previous valid SMILES with the best logCMC and retry Steps 3–4.
Iteration Details:

Iteration 0: Use the initial input SMILES.
Iteration Limit: MUST only stop after generating 10 new SMILES candidates (10 new SMILES MUST be in your final answer).
Molecular Weight: Prioritize changes that keep the molecular weight below 600 g/mol.
Final Steps:

Document All Changes: Even if reverted or invalid SMILES, all changes must be documented and reported in the end given the format instructions.
Avoid Redundancy: Do not repeat SMILES in the iteration history.
Begin with Problem Description:
Question: {input}
Thought: {agent_scratchpad}
"""

In [6]:
from dziner.agents import dZiner
from langchain_anthropic import ChatAnthropic

agent_model = ChatAnthropic(model="claude-3-5-sonnet-20240620", api_key=os.environ["ANTHROPIC_API_KEY"],
                      temperature=0.3, max_tokens=8192)


agent = dZiner(tools, property="Critical Micelle Concentration (CMC)",
               model=agent_model, verbose=True, n_design_iterations=10,
              suffix=SUFFIX, format_instructions=FORMAT_INSTRUCTIONS).agent

In [7]:
response = agent.invoke(input_data)



> Entering new AgentExecutor chain...
Thought: To begin, I need to research the design guidelines for molecules with lower Critical Micelle Concentration (CMC). This will help me understand what changes I should make to the given SMILES to achieve a lower CMC.

Action: domain_knowledge
Action Input: Design guidelines for molecules with lower Critical Micelle Concentration

Observation: The design guidelines for creating surfactants with a specific Critical Micelle Concentration (CMC) involve manipulating the molecular structure, particularly the hydrophobic tails and headgroups. Key strategies include:

1. **Hydrophobic Tail Structure**: The chemical nature and structure of the hydrophobic tails significantly influence the CMC. For instance, fluorocarbon surfactants have lower CMCs due to their larger molecular volume and lower polarizability compared to hydrocarbons. Similarly, branching in hydrocarbon tails can reduce the CMC by increasing the packing density at the interface.

2. 

In [8]:
import json
import pandas as pd

output_str = response['output'].replace("```json", "")[18:-2]
output_dict = json.loads(output_str)
pd.DataFrame(output_dict)

,Iteration,SMILES,Modification,logCMC,uncertainty,SA Score,Molecular Weight,Validity
0,0,CCCCCCCCCC(=O)CC(=O)NC1CCOC1=O,Initial SMILES,1.785,0.321,2.80,297.40,Valid
1,1,CCCCCCCCCCCC(=O)CC(=O)NC1CCOC1=O,Extended the alkyl chain by adding two carbon ...,1.054,0.268,2.81,325.45,Valid
2,2,CCC(C)CCCCCCCC(=O)CC(=O)NC1CCOC1=O,Introduced branching in the hydrophobic tail b...,2.225,0.369,3.22,325.45,Valid
3,3,CCCCCCCCCCCC(=O)CC(=O)Nc1ccccc1,Modified the head group by replacing the cycli...,2.334,0.422,1.85,317.47,Valid
4,4,CCCCCCCCCCCOCC(=O)CC(=O)NC1CCOC1=O,Introduced an ether linkage in the hydrophobic...,0.798,0.345,2.93,355.48,Valid
5,5,FCCCCCCCCCCC(=O)CC(=O)NC1CCOC1=O,Added a fluorine atom at the end of the hydrop...,-0.779,0.670,3.02,329.41,Valid
6,6,FCCCCCCCCCCC(=O)C(C)C(=O)NC1CCOC1=O,Introduced a branched structure near the head ...,-1.109,0.515,3.38,343.44,Valid
7,7,FCCCCCCCCCCF(=O)C(C)C(=O)NC1CCOC1=O,"Attempted to add a second fluorine atom, but r...",NaN,NaN,NaN,NaN,Invalid
8,8,FCCCCCCC#CCC(=O)C(C)C(=O)NC1CCOC1=O,Introduced a triple bond in the hydrophobic tail,0.065,0.737,3.88,325.38,Valid
9,9,FCCCCCCCC(O)CCC(=O)C(C)C(=O)NC1CCOC1=O,Introduced a hydroxyl group in the hydrophobic...,0.032,0.808,3.77,359.44,Valid
